In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

## 2D meshing example

This notebook shows how to mesh a 2D image:

1. Load and visualize a volume
2. Select a plane from a volume
3. Apply image filters and segment an image
4. Generate a 2D mesh from the binary image
5. Visualize and save the mesh to gmsh22 format

In [ ]:
from nanomesh.volume import Volume
from nanomesh.plane import Plane
import pyvista as pv
from skimage import filters
import numpy as np

In [ ]:
vol = Volume.load('sample_data.npy')

In [ ]:
vol.show_volume()

In [ ]:
vol.show_slice(x=123)

In [ ]:
plane = vol.select_plane(x=161)
plane.show()

In [ ]:
plane_gauss = plane.apply(filters.gaussian, sigma=5)
plane_gauss.show()

In [ ]:
filters.try_all_threshold(plane_gauss.image, figsize=(5, 10), verbose=False);

In [ ]:
thresh = filters.threshold_li(plane_gauss.image)
thresh

In [ ]:
from nanomesh.mesh2d import Mesher2D

mesher = Mesher2D(plane_gauss.image)
mesher.generate_contours(label=1, max_contour_dist = 5, level=thresh)

mesh = mesher.triangulate(label=1, opts='q30a100', plot=True)

In [ ]:
mesh.plot_itk()

Save the output...

In [ ]:
vmesh.write('out.msh', file_format='gmsh22', binary=False)